In [ ]:
!pip install slack_sdk
!pip install qdrant_client
!pip install duckduckgo-search

In [ ]:

from dotenv import find_dotenv, load_dotenv

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

load_dotenv()
load_dotenv(dotenv_path=find_dotenv(filename=".env.local"))

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

qdrant_client = QdrantClient(host="localhost", port=6333)
qdrant = Qdrant(
    client=qdrant_client,
    collection_name="SlackMessages",
    content_payload_key="text",
    embeddings=embeddings,
)
retriever = qdrant.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
prompt_template = """Use the following nodes of the user's Graph to answer/fulfil the inquiry at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Do not reply with raw IDs that would be meaningless to users. Treat the context entities as information you know, and not information the user can/should see. Answer precisely and concisely.
Graph: ---
{context}
---
Inquiry: {question}
Helpful Answer:"""
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
retrieval_qa = RetrievalQA.from_chain_type(llm, retriever=retriever)

In [ ]:
from langchain.agents import initialize_agent, Tool, tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

# @tool
# def bail(query: str) -> str:
#     """Bail out of the conversation."""
#     return f"Final answer is that you do not know the answer to the question."

tools = [
    Tool(
        name="Search",
        func=retriever.get_relevant_documents,
        description="use this tool initially to gain context used for follow-up lookups",
    ),
    Tool(
        name="Lookup",
        func=search,
        description="the only tool for searching the internet for facts or uncovering missing information",
    ),
    # Tool(
    #     name="Bail",
    #     func=bail,
    #     description="in case the other tools are not helpful, use this tool to bail out of the conversation, provide an empty action input",
    # ),
]

react = initialize_agent(
    tools,
    llm,
    early_stopping_method="generate",
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
react.run("")